In [1]:
import logging

import numpy

from cicada import transcript
from cicada.arithmetic import field

In [2]:
with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.code_handler())
    
    f = field(order=127)
    a = f.ones(3)
    b = f.uniform(size=3, generator=numpy.random.default_rng())
    a += b

cicada.transcript.assert_equal(cicada.arithmetic.field(order=127).ones(shape=3), numpy.array([1, 1, 1], dtype='object'))
bg = numpy.random.PCG64()
bg.state = {'bit_generator': 'PCG64', 'state': {'state': 3110400654694539912723698677021134952, 'inc': 2627829065645458385012291873148098333}, 'has_uint32': 0, 'uinteger': 0}
cicada.transcript.assert_equal(cicada.arithmetic.field(order=127).uniform(size=3, generator=numpy.random.Generator(bg)), numpy.array([3, 83, 11], dtype='object'))

lhs = numpy.array([1, 1, 1], dtype='object')
lhs += numpy.array([3, 83, 11], dtype='object')
cicada.transcript.assert_equal(lhs, numpy.array([4, 84, 12], dtype='object'))



cicada.transcript.record
cicada.transcript.code_handler
cicada.transcript.Formatter.__init__
cicada.transcript.Formatter.__init__
cicada.transcript.code_handler
cicada.transcript.set_handler
cicada.transcript.set_handler
cicada.arithmetic.field
cicada.arithmetic.field.<locals>.FieldMeta.__new__
cicada.arithmetic.field.<locals>.FieldMeta.__new__
cicada.arithmetic.field
cicada.arithmetic.field.<locals>.FieldMeta.ones
cicada.arithmetic.field.<locals>.FieldMeta.ones
cicada.arithmetic.field.<locals>.FieldMeta.uniform
cicada.arithmetic.field.<locals>.FieldMeta.bytes
cicada.arithmetic.field.<locals>.FieldMeta.bits
cicada.arithmetic.field.<locals>.FieldMeta.bits
cicada.arithmetic.field.<locals>.FieldMeta.bytes
cicada.arithmetic.field.<locals>.FieldMeta.bytes
cicada.arithmetic.field.<locals>.FieldMeta.bits
cicada.arithmetic.field.<locals>.FieldMeta.bits
cicada.arithmetic.field.<locals>.FieldMeta.bytes
cicada.arithmetic.field.<locals>.FieldMeta.bytes
cicada.arithmetic.field.<locals>.FieldMeta.bi